In [70]:
cd Desktop

[WinError 2] The system cannot find the file specified: 'Desktop'
C:\Users\sanke\Desktop


In [71]:
import cv2
import time 
import numpy as np
import os


In [72]:
# Function to detect faces in a photo and return the faces along with the gray image.
def faceDetector(img):
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_classifier = cv2.CascadeClassifier(r'C:\Users\sanke\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
    faces = face_classifier.detectMultiScale(gray_img,scaleFactor=1.2,minNeighbors=5)
    return faces,gray_img

In [73]:
def CreateTrainingData(directory):
    face=[]
    face_labels=[]
    for path,subdir,filenames in os.walk(directory):
        for filename in filenames:
            img_path=os.path.join(path,filename)
            face_id = os.path.basename(path)
            img = cv2.imread(img_path)
            faces,gray_img = faceDetector(img)
            if len(faces)>1:
                continue
            if img is None:
                continue
            (x,y,w,z) = faces[0]
            roi_img = img[y:y+z,x:x+w]
            np.append(faces,roi_img)
            np.append(face_labels,face_id)
    return face,face_labels

In [74]:
def classifier(face,face_labels):
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(face,face_labels)
    return face_recognizer

In [75]:
def DrawRectangle(img,face):
    (x,y,w,z) = face[0]
    cv2.rectangle(img,(x,y),(x+w,y+z),(255,0,0),5)

In [76]:
def put_text(test_img,text,x,y):
    cv2.putText(test_img,text,(x,y),cv2.FONT_HERSHEY_DUPLEX,2,(255,0,0),4)

In [77]:
faces,faceID=CreateTrainingData('FaceRecognisor')
face_recognizer=classifier(faces,faceID)
face_recognizer.write('trainingData.yml')

IndexError: tuple index out of range

In [ ]:
video = cv2.VideoCaptire(0)
time.sleep(10)
while True:
    check,frame = video.read()
    face,gray_frame = faceDetector(frame)
    (x,y,w,z)=face[0]
    gray_frame = gray_frame[x:x+w,y:y+z]
    label = face_recognizer.predict(gray_frame)
    names = [0:'unknown',1:'Sanket']
    txt = names[label]
    put_text(frame,txt,x,y)
    DrawRectangle(frame,face)
    cv2.imshow('person',frame)
    k=cv2.waitKey(0)
    if k==ord('q'):
        break
video.release()
cv2.destroyAllWindows()